In [1]:
import cobra

In [3]:
FBA_model = cobra.io.read_sbml_model('H_volcanii_biomass_replaced2.sbml')

In [57]:
## Function to reverse a reaction: products become reactants and vice versa. 
## Because of the way Cobra keeps track of stoichiometry, the easiest way is probably to add -2 times the reaction to itself
def reverse(rxn):
    stoich = rxn.metabolites
    minus_two = {}
    for met,coeff in stoich.items():
        minus_two[met] = -2*coeff
    rxn.add_metabolites(minus_two)

In [5]:
## reverse tetrahydrodipicolinate succinylation reaction
react = FBA_model.reactions.get_by_id('rxn03031_c0')
reverse(react)

In [6]:
## reverse dihydroneopterin aldolase reaction
react = FBA_model.reactions.get_by_id('rxn01152_c')
reverse(react)

In [7]:
print(FBA_model.reactions.get_by_id('rxn03031_c0'))

rxn03031_c0: cpd00001_c0 + cpd00078_c0 + cpd02465_c0 --> cpd00010_c0 + cpd02724_c0


In [8]:
print(FBA_model.reactions.get_by_id('rxn01152_c'))

rxn01152_c: cpd02961_c0 --> cpd00229_c0 + cpd00954_c0


In [9]:
## Correct ID for menaquinone-8
cofactors_rxn = FBA_model.reactions.get_by_id('rxn_cofactors')
to_remove = FBA_model.metabolites.get_by_id('cpd11606_c0')
coeff = cofactors_rxn.get_coefficient(to_remove)
to_add = FBA_model.metabolites.get_by_id('cpd15500_c0')
cofactors_rxn.subtract_metabolites({to_remove: coeff})
cofactors_rxn.add_metabolites({to_add: coeff})

In [13]:
print(cofactors_rxn.build_reaction_string(use_metabolite_names=True))

3.32 ATP_c0 + 0.04 NAD_c0 + 0.04 NADP_c0 + 0.04 CoA_c0 + 0.04 FAD_c0 + 0.04 Acetyl_CoA_c0 + 49.82 L_Glutamate_c0 + 0.07 Tetrahydrofolate_c0 + 0.02 Calomide_c0 + 0.06 10_Formyltetrahydrofolate_c0 + 0.11 Thiamin_c0 + 0.06 5_10_Methenyltetrahydrofolate_c0 + 0.03 Siroheme_c0 + 0.01 Menaquinone_8_c0 --> Cofactors


In [31]:
## set medium
med_components = ['cpd00190_e0','cpd00013_e0','cpd00007_e0','cpd00009_e0','cpd00971_e0','cpd00048_e0','cpd10516_e0','cpd00149_e0','cpd00067_e0','cpd00001_e0','cpd00305_e0','cpd00104_e0']
med = {}

for component in med_components:
	try:
		met = FBA_model.metabolites.get_by_id(component)
	except:
		print('model to be filled has no ' + component + ': Adding it!')
		met = cobra.Metabolite(component)
	try:
		exchange = FBA_model.reactions.get_by_id('EX_' + component)
	except:
		exchange = FBA_model.add_boundary(met)
		print('new exchange reaction added: EX_' + component)
	exch_id = exchange.id
	med[exch_id] = 300
		
FBA_model.medium = med

In [20]:
 ## retry biomass components now
def try_as_biomass(met):
    react_dict = {}
    test_biomass = FBA_model.reactions.get_by_id('test_biomass')
    biomass = FBA_model.metabolites.get_by_id('cpd11416_c0')
    react_dict[biomass] = 1
    react_dict[met] = -1
    old_mets = test_biomass.metabolites
    test_biomass.subtract_metabolites(old_mets)
    test_biomass.add_metabolites(react_dict)
    FBA_model.objective = {FBA_model.reactions.test_biomass: 1}

    solution = FBA_model.optimize()
    if solution.f > 0.0001:
        return 'Success!'
    else:
        return 'Failed :-('

test_biomass = cobra.Reaction('test_biomass')
FBA_model.add_reaction(test_biomass)

biomass_rxn = FBA_model.reactions.get_by_id('biomass0')
for met in biomass_rxn.metabolites:
    if not met.id == 'cpd11416_c0':
        print(str(met.id) + ': ' + try_as_biomass(met))

Ignoring reaction 'test_biomass' since it already exists.


DNA_c0: Success!
Energy_c0: Success!
Glycosylation_c0: Success!
Lipid_c0: Success!
Protein_c0: Success!
RNA_c0: Success!
Salt_c0: Success!
cofactors_c0: Success!


In [18]:
## So only the requirement to take up salt is keeping it from growing now. 
## This is weird because taking in salt can be coupled to proton export, whose re-entry can be coupled to ATP synthesis.
## This should actually help the cell grow--but maybe the problem is it has no way to use the excess ATP 
## when biomass is ONLY salt (is that possible?)
## So try the regular biomass again just to be sure
FBA_model.objective = {biomass_rxn: 1}
solution = FBA_model.optimize()
print(solution.f)

0.0


In [21]:
## Realized there was no sodium in the media!! Duh! Adding it above and running the above again
FBA_model.objective = {biomass_rxn: 1}
solution = FBA_model.optimize()
print(solution.f)

0.0720037441946975


In [32]:
### So it grows on glucose now (slowly it seems). Boundary fluxes are arbitrarily set at 300 for both glucose and ammonia
## Since this is 6:1 C:N ratio, either one may be limiting. So this corresponds to about 4300 N/biomass unit or 26,000 C/unit
## (including energy need). It shouldn't grow on fructose yet since there's no fructose bisphosphate aldolase, let's see
glucose_extra = 'EX_cpd00190_e0'
fructose_extra = 'EX_cpd00082_e0'
med[glucose_extra] = 0
med[fructose_extra] = 300

FBA_model.medium = med

solution = FBA_model.optimize()
print(solution.f)

0.07200374419469768


In [35]:
## It actually does, and I just realized why... it reverses the upper part of glycolysis and then uses the E-D pathway
## Now add the aldolase though

## Re-load the saved updated model from above
FBA_model = cobra.io.read_sbml_model('H_volcanii_correct.sbml')

In [36]:
stoich = {}
aldolase_rxn = cobra.Reaction('rxn_01068_KEGG', name='Fructose bisphosphate aldolase')
FBP = FBA_model.metabolites.get_by_id('cpd00290_c0')
GAP = FBA_model.metabolites.get_by_id('cpd00102_c0')
DHAP = FBA_model.metabolites.get_by_id('cpd00095_c0')
stoich[FBP] = -1
stoich[GAP] = 1
stoich[DHAP] = 1
aldolase_rxn.add_metabolites(stoich)
## Assign gene
aldolase_rxn.gene_reaction_rule = 'HVO_0790'
## Make it reversible, as we know that this step can go in both directions in most organisms
aldolase_rxn.lower_bound = -1 * aldolase_rxn.upper_bound
FBA_model.add_reaction(aldolase_rxn)

In [37]:
## set medium
med_components = ['cpd00190_e0','cpd00013_e0','cpd00007_e0','cpd00009_e0','cpd00971_e0','cpd00048_e0','cpd10516_e0','cpd00149_e0','cpd00067_e0','cpd00001_e0','cpd00305_e0','cpd00104_e0']
med = {}

for component in med_components:
	try:
		met = FBA_model.metabolites.get_by_id(component)
	except:
		print('model to be filled has no ' + component + ': Adding it!')
		met = cobra.Metabolite(component)
	try:
		exchange = FBA_model.reactions.get_by_id('EX_' + component)
	except:
		exchange = FBA_model.add_boundary(met)
		print('new exchange reaction added: EX_' + component)
	exch_id = exchange.id
	med[exch_id] = 300
		
FBA_model.medium = med

In [38]:
## Try fructose again
glucose_extra = 'EX_cpd00190_e0'
fructose_extra = 'EX_cpd00082_e0'
med[glucose_extra] = 0
med[fructose_extra] = 300

FBA_model.medium = med

solution = FBA_model.optimize()
print(solution.f)

0.07200374419469824


In [10]:
## Since it grows no faster, I'm guessing nitrogen is limiting. Display limiting fluxes...

for reaction in FBA_model.reactions:
	flux = solution.x_dict[reaction.id]
	if flux == reaction.lower_bound or flux == reaction.upper_bound:
		if not flux == 0:
			print(reaction.id)
			print(reaction.build_reaction_string(use_metabolite_names = True))
			print(flux)

rxn03541_c0
1_Aminopropan_2_ol_c0 + Adenosylcobyric_acid_c0 <=> H2O_c0 + Adenosyl_cobinamide_c0
1000.0
rxn05054_c0
ATP_c0 + 1_Aminopropan_2_ol_c0 + Adenosylcobyric_acid_c0 <=> ADP_c0 + Phosphate_c0 + H_c0 + Adenosyl_cobinamide_c0
-1000.0
EX_cpd00082_e0
D_Fructose_e0 <=> 
-300.0
EX_cpd00013_e0
NH3_e0 <=> 
-300.0
rxn_atp_syn
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
1000.0


In [39]:
## There is a thermodynamically impossible loop involving an amide ligase from the B12 pathway 
## generating a bunch of surplus ATP
## What happens if the ATP-consuming reaction is set as irreversible?
amide_ligase = FBA_model.reactions.get_by_id('rxn05054_c0')
amide_ligase.lower_bound = 0



In [40]:
solution = FBA_model.optimize()
print(solution.f)

0.0720037441946987


In [13]:
## Hmm... doesn't grow any slower! Maybe there's another loop now.
## Limiting fluxes?
for reaction in FBA_model.reactions:
	flux = solution.x_dict[reaction.id]
	if flux == reaction.lower_bound or flux == reaction.upper_bound:
		if not flux == 0:
			print(reaction.id)
			print(reaction.build_reaction_string(use_metabolite_names = True))
			print(flux)


rxn00519_c0
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0
rxn00515_c0
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
EX_cpd00082_e0
D_Fructose_e0 <=> 
-300.0
EX_cpd00013_e0
NH3_e0 <=> 
-300.0
rxn_atp_syn
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
1000.0


In [41]:
## I see the problem--The stoichiometry of ATP synthase per glucose oxidized is around 30:1
## Therefore, its upper bound must be set very high, otherwise it limits respiration
## Set it to 10,0000
## At ~33 times the sugar uptake bound, even if 100% is devoted to respiration this won't limit now
atp_syn = FBA_model.reactions.get_by_id('rxn_atp_syn')
atp_syn.upper_bound = 10000

In [16]:
solution = FBA_model.optimize()
print(solution.f)

0.07200374419469883


In [17]:
for reaction in FBA_model.reactions:
	flux = solution.x_dict[reaction.id]
	if flux == reaction.lower_bound or flux == reaction.upper_bound:
		if not flux == 0:
			print(reaction.id)
			print(reaction.build_reaction_string(use_metabolite_names = True))
			print(flux)

rxn09240_c0
H2O_c0 + ATP_c0 + GTP_c0 + Sulfate_c0 <=> Phosphate_c0 + PPi_c0 + GDP_c0 + H_c0 + APS_c0
-1000.0
rxn00519_c0
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0
rxn10113_c0
0.5 O2_c0 + 2.5 H_c0 + Ubiquinol_8_c0 --> H2O_c0 + 2.5 H_e0 + Ubiquinone_8_c0
1000.0
rxn00058_c0
O2_c0 + 4.0 H_c0 + 4.0 Cytochrome_c2_c0 <=> 2.0 H2O_c0 + 4.0 Cytochrome_c3_c0
-1000.0
rxn10122_c0
NADH_c0 + 4.5 H_c0 + Ubiquinone_8_c0 <=> NAD_c0 + 3.5 H_e0 + Ubiquinol_8_c0
1000.0
rxn00515_c0
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
rxn00379_c0
ATP_c0 + Sulfate_c0 <=> PPi_c0 + APS_c0
1000.0
rxn10043_c0
0.5 O2_c0 + 6.0 H_c0 + 2.0 Cytochrome_c2_c0 <=> H2O_c0 + 4.0 H_e0 + 2.0 Cytochrome_c3_c0
1000.0
EX_cpd00082_e0
D_Fructose_e0 <=> 
-300.0
EX_cpd00013_e0
NH3_e0 <=> 
-300.0
EX_cpd00009_e0
Phosphate_e0 <=> 
-300.0


In [42]:
## Now the electron transport chain is limiting. This will take work to fix.
## For now, just set nutrient uptake really low
## At 12 reducing equivalents per glucose, anything below 1000/12 ~83 should be fine
## Leave the nitrogen for now, and set fructose to 50. At 1:1 C:N, carbon should certainly limit
med['EX_cpd00082_e0'] = 20
FBA_model.medium = med

In [43]:
solution = FBA_model.optimize()
print(solution.f)

0.007379887715008395


In [25]:
for reaction in FBA_model.reactions:
	flux = solution.x_dict[reaction.id]
	if flux == reaction.lower_bound or flux == reaction.upper_bound:
		if not flux == 0:
			print(reaction.id)
			print(reaction.build_reaction_string(use_metabolite_names = True))
			print(flux)

rxn09240_c0
H2O_c0 + ATP_c0 + GTP_c0 + Sulfate_c0 <=> Phosphate_c0 + PPi_c0 + GDP_c0 + H_c0 + APS_c0
-1000.0
rxn00519_c0
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0
rxn00499_c0
NAD_c0 + L_Lactate_c0 <=> NADH_c0 + Pyruvate_c0 + H_c0
1000.0
rxn00515_c0
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
rxn00379_c0
ATP_c0 + Sulfate_c0 <=> PPi_c0 + APS_c0
1000.0
rxn10043_c0
0.5 O2_c0 + 6.0 H_c0 + 2.0 Cytochrome_c2_c0 <=> H2O_c0 + 4.0 H_e0 + 2.0 Cytochrome_c3_c0
1000.0
EX_cpd00082_e0
D_Fructose_e0 <=> 
-20.0
EX_cpd00009_e0
Phosphate_e0 <=> 
-300.0


In [26]:
## Something is clearly wrong--with only 20 moles of sugar, where are 1000 moles of lactate coming from??
## Model seems to be fixing CO2 through reversal of PEPCK reaction, which is clearly unrealistic
## because fixing CO2 (high entropy gas to aqueous) *requires* free energy rather than producing it
## This also doesn't explain the lactate source

## It's also phosphate limited now, which is of course strange because phosphate is MORE abundant now relative to carbon
## Try loopless version of FBA:

cobra.flux_analysis.loopless.add_loopless(FBA_model)
solution = FBA_model.optimize()
print(solution.f)

0.007379887712794733


In [27]:
for reaction in FBA_model.reactions:
	flux = solution.x_dict[reaction.id]
	if flux == reaction.lower_bound or flux == reaction.upper_bound:
		if not flux == 0:
			print(reaction.id)
			print(reaction.build_reaction_string(use_metabolite_names = True))
			print(flux)

rxn00519_c0
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0
rxn01705_c0
dCTP_c0 + Cytidine_c0 <=> CMP_c0 + H_c0 + dCDP_c0
-1000.0
rxn05142_c0
H2O_c0 + NAD_c0 + Sinapaldehyde_c0 <=> NADH_c0 + 2.0 H_c0 + Sinapate_c0
-1000.0
rxn05298_c0
L_Glutamate_e0 + Na_e0 <=> L_Glutamate_c0 + Na_c0
1000.0
rxn01516_c0
Uridine_c0 + TTP_c0 <=> H_c0 + UMP_c0 + dTDP_c0
-1000.0
rxn00707_c0
ITP_c0 + Cytidine_c0 <=> CMP_c0 + H_c0 + IDP_c0
1000.0
rxn00237_c0
ATP_c0 + GDP_c0 <=> ADP_c0 + GTP_c0
1000.0
rxn05143_c0
H2O_c0 + NADP_c0 + Sinapaldehyde_c0 <=> NADPH_c0 + 2.0 H_c0 + Sinapate_c0
1000.0
rxn00184_c0
H2O_c0 + NADP_c0 + L_Glutamate_c0 <=> NADPH_c0 + NH3_c0 + 2_Oxoglutarate_c0 + H_c0
-1000.0
rxn00368_c0
UTP_c0 + Cytidine_c0 <=> UDP_c0 + CMP_c0 + H_c0
-1000.0
rxn05297_c0
L_Glutamate_e0 + H_e0 <=> L_Glutamate_c0 + H_c0
-1000.0
EX_cpd00082_e0
D_Fructose_e0 <=> 
-20.0


In [34]:
## That didn't do anything in terms of growth rate
## It seems add_loopless may only detect "absolute" loops, with no net change AT ALL
## i.e. A --> B and then B --> A (by different paths)
## not "reverse futile cycles" like A + ADP --> B + ATP and then B --> A by another path, which is what I want
## The former don't affect growth rate anyway so I don't know why one would care

## I may need to write my own algorithm for "reverse futile cycles" later--it's probably needs a graph algorithm
## rather that something that can be done while staying within the framework of linear optimization

## For now, let's see how this makes ATP (or GTP, etc.):

atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

ATP_c0 + CMP_c0 + H_c0 <=> ADP_c0 + CDP_c0
-365.915348941
ATP_c0 + dCDP_c0 <=> ADP_c0 + dCTP_c0
-999.967307097
ATP_c0 + D_Ribulose_c0 <=> ADP_c0 + H_c0 + D_Ribulose5_phosphate_c0
-2.34213036498
ATP_c0 + AMP_c0 + H_c0 <=> 2.0 ADP_c0
-2.98766143863
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.597254312584
ATP_c0 + beta_D_Glucose_c0 <=> ADP_c0 + H_c0 + beta_D_Glucose_6_phosphate_c0
-17.5241460697
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-365.948041843
ATP_c0 + dADP_c0 <=> ADP_c0 + dATP_c0
-365.235661282
ATP_c0 + GDP_c0 <=> ADP_c0 + GTP_c0
1000.0
UTP_c0 + Cytidine_c0 <=> UDP_c0 + CMP_c0 + H_c0
-1000.0
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [44]:
## Odd! ATP synthase is not even in there! This is worse than without the loopless condition (which can't be removed)
## Let's reset the model and re-do the above:

atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

ATP_c0 + dCDP_c0 <=> ADP_c0 + dCTP_c0
-49.7749977937
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-50.8959289388
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.597254312776
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-0.0321025115603
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
2065.58858813
H2O_c0 + ATP_c0 + GTP_c0 + Sulfate_c0 <=> Phosphate_c0 + PPi_c0 + GDP_c0 + H_c0 + APS_c0
-1.58099334519
UTP_c0 + Uridine_c0 <=> UDP_c0 + H_c0 + UMP_c0
-1.05525014437
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [45]:
## Now ATP synthase makes a lot of ATP, as expected. But there are other problems. 
## Let's re-solve the model and see if these anomalies are consistent:

solution = FBA_model.optimize()
print(solution.f)

0.007379887715008395


In [46]:
atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

ATP_c0 + dCDP_c0 <=> ADP_c0 + dCTP_c0
-49.7749977937
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-50.8959289388
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.597254312776
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-0.0321025115603
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
2065.58858813
H2O_c0 + ATP_c0 + GTP_c0 + Sulfate_c0 <=> Phosphate_c0 + PPi_c0 + GDP_c0 + H_c0 + APS_c0
-1.58099334519
UTP_c0 + Uridine_c0 <=> UDP_c0 + H_c0 + UMP_c0
-1.05525014437
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [47]:
## So the ATP synthase flux is consistent, and the PEPCK too, but other things vary
## How about cation efflux, what's driving that?

H_in = 'cpd00067_c0'
H_out = 'cpd00067_e0'
Na_in = 'cpd00971_c0'
Na_out = 'cpd00971_e0'

pairs =[(H_in,H_out), (Na_in,Na_out)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])


PPi_c0 + H_c0 --> PPi_e0 + H_e0
148.703353728
0.5 O2_c0 + 2.5 H_c0 + Ubiquinol_8_c0 --> H2O_c0 + 2.5 H_e0 + Ubiquinone_8_c0
662.430286583
NADH_c0 + 4.5 H_c0 + Ubiquinone_8_c0 <=> NAD_c0 + 3.5 H_e0 + Ubiquinol_8_c0
661.89214517
H_e0 + Acetoacetate_e0 <=> H_c0 + Acetoacetate_c0
-1.84179857703
H_e0 + L_Lactate_e0 <=> H_c0 + L_Lactate_c0
-2.88422497691e-13
H_c0 + Na_e0 <=> H_e0 + Na_c0
22.4699869191
0.5 O2_c0 + 6.0 H_c0 + 2.0 Cytochrome_c2_c0 <=> H2O_c0 + 4.0 H_e0 + 2.0 Cytochrome_c3_c0
1000.0


In [49]:
## Even weirder... so at the level of entering the electron transport chain from NADH there are ~1300 electrons
## but there's ~1300 being used to reduce O2 at the quinone stage and another 1000 at cytochrome C oxidase!
## Where are all these electrons to cytochrome C coming from??

cyt_c_ox = 'cpd00109_c0'
cyt_c_red = 'cpd00110_c0'

reactant = FBA_model.metabolites.get_by_id(cyt_c_ox)
product = FBA_model.metabolites.get_by_id(cyt_c_red)
for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if reactant in met_dict and product in met_dict:
        if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00058_c0
O2_c0 + 4.0 H_c0 + 4.0 Cytochrome_c2_c0 <=> 2.0 H2O_c0 + 4.0 Cytochrome_c3_c0
-829.183128108


In [50]:
## So Cyt. C oxidase is in there twice, going in opposite directions! The proton stoichiometry is different, though. 
## This creates a proton-pumping cycle with no free energy source.
## Only plant photosystems can oxidize water, it can't be done here. So set this other oxidase as irreversible consuming O2.
## That will solve this problem.

oxidase = FBA_model.reactions.get_by_id('rxn00058_c0')
oxidase.lower_bound = 0
solution = FBA_model.optimize()
print(solution.f)



0.007379887715008557


In [51]:
H_in = 'cpd00067_c0'
H_out = 'cpd00067_e0'
Na_in = 'cpd00971_c0'
Na_out = 'cpd00971_e0'

pairs =[(H_in,H_out), (Na_in,Na_out)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

PPi_c0 + H_c0 --> PPi_e0 + H_e0
148.703353728
0.5 O2_c0 + 2.5 H_c0 + Ubiquinol_8_c0 --> H2O_c0 + 2.5 H_e0 + Ubiquinone_8_c0
872.368809382
H_e0 + XAN_e0 <=> H_c0 + XAN_c0
-1.43288159116e-14
NADH_c0 + 4.5 H_c0 + Ubiquinone_8_c0 <=> NAD_c0 + 3.5 H_e0 + Ubiquinol_8_c0
871.83066797
H_e0 + Acetoacetate_e0 <=> H_c0 + Acetoacetate_c0
-1.84179857703
H_c0 + Na_e0 <=> H_e0 + Na_c0
22.4699869191
2.0 H_c0 + Nitrate_c0 + Menaquinol_8_c0 <=> H2O_c0 + 2.0 H_e0 + Nitrite_c0 + Menaquinone_8_c0
8.21730109605e-33


In [52]:
## Now where is all that PPi coming from?
ppi = FBA_model.metabolites.get_by_id('cpd00012_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if ppi in met_dict:
        if met_dict[ppi] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[ppi] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn12008_c0
Isopentenyldiphosphate_c0 + all_trans_Heptaprenyl_diphosphate_c0 --> PPi_c0 + H_c0 + Farnesylfarnesylgeraniol_c0
7.37988771501e-05
rxn09177_c0
CTP_c0 + L_Cysteine_c0 + 4_phosphopantothenate_c0 --> PPi_c0 + CMP_c0 + 2.0 H_c0 + R_4_Phosphopantothenoyl_L_cysteine_c0
0.0005903910172
rxn01486_c0
Isopentenyldiphosphate_c0 + Farnesyldiphosphate_c0 --> PPi_c0 + H_c0 + Geranylgeranyl_diphosphate_c0
0.376743267851
rxn00416_c0
H2O_c0 + ATP_c0 + L_Aspartate_c0 + L_Glutamine_c0 --> PPi_c0 + AMP_c0 + L_Glutamate_c0 + 2.0 H_c0 + L_Asparagine_c0
116.391327879
rxn00789_c0
ATP_c0 + PRPP_c0 --> PPi_c0 + H_c0 + Phosphoribosyl_ATP_c0
0.597844703793
rxn01362_c0
PRPP_c0 + Orotate_c0 --> PPi_c0 + H_c0 + Orotidylic_acid_c0
0.975547357047
rxn03891_c0
Isopentenyldiphosphate_c0 + all_trans_Hexaprenyl_diphosphate_c0 --> PPi_c0 + H_c0 + all_trans_Heptaprenyl_diphosphate_c0
7.37988771501e-05
rxn06937_c0
ATP_c0 + L_Glutamate_c0 + tRNA_Glu_c0 --> PPi_c0 + AMP_c0 + H_c0 + L_Glutamyl_tRNA_Glu_c0
0.0029519550

In [53]:
## What's making NTPs now?

atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-1.12093114503
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.597254312776
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-0.0321025115603
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
512.191593317
H2O_c0 + ATP_c0 + GTP_c0 + Sulfate_c0 <=> Phosphate_c0 + PPi_c0 + GDP_c0 + H_c0 + APS_c0
-1.58099334519
UTP_c0 + Uridine_c0 <=> UDP_c0 + H_c0 + UMP_c0
-1.05525014437
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [54]:
## Now ATP synthase is reasonable--it's only making 25.61 ATP/fructose
## But where's all that PEP coming from?

pep = FBA_model.metabolites.get_by_id('cpd00061_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if pep in met_dict:
        if met_dict[pep] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[pep] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00147_c0
H2O_c0 + ATP_c0 + Pyruvate_c0 --> Phosphate_c0 + AMP_c0 + Phosphoenolpyruvate_c0 + 3.0 H_c0
131.483183786
rxn00251_c0
Phosphate_c0 + Oxaloacetate_c0 + H_c0 <=> H2O_c0 + CO2_c0 + Phosphoenolpyruvate_c0
824.076448474
rxn00745_c0
Phosphoenolpyruvate_c0 + Glycerone_c0 <=> Pyruvate_c0 + Glycerone_phosphate_c0
-34.1778835835
rxn00459_c0
2_Phospho_D_glycerate_c0 <=> H2O_c0 + Phosphoenolpyruvate_c0
33.1878716465


In [75]:
## It's using PEP carboxylase in reverse to MAKE PEP, although this reaction is irreversible (deltaG = -30 kJ/mol)
## Reverse it, then set to irreversible:

pepc = FBA_model.reactions.get_by_id('rxn00251_c0')
reverse(pepc)
pepc.lower_bound = 0
solution = FBA_model.optimize()
print(solution.f)

0.007379887715008347


In [59]:
## What's making NTPs now?

atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

ATP_c0 + CMP_c0 + H_c0 <=> ADP_c0 + CDP_c0
-2.2914668279e-17
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-1.09871768301
H2O_c0 + ATP_c0 + L_Leucine_e0 --> ADP_c0 + Phosphate_c0 + H_c0 + L_Leucine_c0
-1.38777878078e-17
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.597254312776
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-0.0326929025775
H2O_c0 + ATP_c0 + met_L_ala_L_e0 --> ADP_c0 + Phosphate_c0 + H_c0 + met_L_ala_L_c0
-1.11022302463e-16
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
512.303554896
H2O_c0 + ATP_c0 + GTP_c0 + Sulfate_c0 <=> Phosphate_c0 + PPi_c0 + GDP_c0 + H_c0 + APS_c0
-825.470839188
UTP_c0 + Uridine_c0 <=> UDP_c0 + H_c0 + UMP_c0
-1.05525014437
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [60]:
pep = FBA_model.metabolites.get_by_id('cpd00061_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if pep in met_dict:
        if met_dict[pep] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[pep] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00305_c0
Oxaloacetate_c0 + GTP_c0 --> CO2_c0 + GDP_c0 + Phosphoenolpyruvate_c0
823.889845842
rxn00147_c0
H2O_c0 + ATP_c0 + Pyruvate_c0 --> Phosphate_c0 + AMP_c0 + Phosphoenolpyruvate_c0 + 3.0 H_c0
131.669786418
rxn00745_c0
Phosphoenolpyruvate_c0 + Glycerone_c0 <=> Pyruvate_c0 + Glycerone_phosphate_c0
-34.1778835835
rxn00459_c0
2_Phospho_D_glycerate_c0 <=> H2O_c0 + Phosphoenolpyruvate_c0
33.1878716465


In [69]:
## The only thing fishy left here is the one with DHAP (called "glycerone phosphate" here)
## It takes an alkyl phosphate and makes an acyl phosphate--clearly violating thermodynamics

rxn = FBA_model.reactions.get_by_id('rxn00745_c0')
rxn.lower_bound = 0
solution = FBA_model.optimize()
print(solution.f)




0.007379887715008391


In [70]:
pep = FBA_model.metabolites.get_by_id('cpd00061_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if pep in met_dict:
        if met_dict[pep] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[pep] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00305_c0
Oxaloacetate_c0 + GTP_c0 --> CO2_c0 + GDP_c0 + Phosphoenolpyruvate_c0
823.882531071
rxn00147_c0
H2O_c0 + ATP_c0 + Pyruvate_c0 --> Phosphate_c0 + AMP_c0 + Phosphoenolpyruvate_c0 + 3.0 H_c0
165.854984772
rxn00745_c0
Phosphoenolpyruvate_c0 + Glycerone_c0 --> Pyruvate_c0 + Glycerone_phosphate_c0
-1.54462698184e-14
rxn00459_c0
2_Phospho_D_glycerate_c0 <=> H2O_c0 + Phosphoenolpyruvate_c0
33.1878716465


In [71]:
## What's making NTPs now?

atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-1.09871768301
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.597254312776
ATP_c0 + 3_Phosphoglycerate_c0 <=> ADP_c0 + 1_3_Bisphospho_D_glycerate_c0
-33.1878716465
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-0.0326929025775
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
512.703948533
H2O_c0 + ATP_c0 + GTP_c0 + Sulfate_c0 <=> Phosphate_c0 + PPi_c0 + GDP_c0 + H_c0 + APS_c0
-826.0824018
UTP_c0 + Uridine_c0 <=> UDP_c0 + H_c0 + UMP_c0
-1.05525014437
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [76]:
## The APS synthesis reaction serves to assimilate sulfate more efficiently when sulfate is scarce
## It's not an energy source

rxn = FBA_model.reactions.get_by_id('rxn09240_c0')
rxn.lower_bound = 0
solution = FBA_model.optimize()
print(solution.f)

0.007379887715008347


In [77]:
atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.id)
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.id)
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

rxn00364_c0
ATP_c0 + CMP_c0 + H_c0 <=> ADP_c0 + CDP_c0
-0.0222134620222
rxn01219_c0
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-1.14314460705
rxn00285_c0
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.597254312776
rxn01100_c0
ATP_c0 + 3_Phosphoglycerate_c0 <=> ADP_c0 + 1_3_Bisphospho_D_glycerate_c0
-33.1878716465
rxn00515_c0
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
rxn00409_c0
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-0.0549063645997
rxn_atp_syn
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
527.662562014
rxn00237_c0
ATP_c0 + GDP_c0 <=> ADP_c0 + GTP_c0
954.963066737
rxn00712_c0
UTP_c0 + Uridine_c0 <=> UDP_c0 + H_c0 + UMP_c0
-1.07746360639
rxn00519_c0
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [78]:
pep = FBA_model.metabolites.get_by_id('cpd00061_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if pep in met_dict:
        if met_dict[pep] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[pep] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00305_c0
Oxaloacetate_c0 + GTP_c0 --> CO2_c0 + GDP_c0 + Phosphoenolpyruvate_c0
952.763196008
rxn00147_c0
H2O_c0 + ATP_c0 + Pyruvate_c0 --> Phosphate_c0 + AMP_c0 + Phosphoenolpyruvate_c0 + 3.0 H_c0
36.9743198357
rxn00459_c0
2_Phospho_D_glycerate_c0 <=> H2O_c0 + Phosphoenolpyruvate_c0
33.1878716465


In [80]:
## What is the rate of citrate synthase?
rxn_id = 'rxn00256_c0'
print(solution.x_dict[rxn_id])


0.0


In [81]:
## Huh? So no flux at all is going into the TCA cycle by this route. Where is NAD being reduced?

NAD = 'cpd00003_c0'
NADH = 'cpd00004_c0'

reactant = FBA_model.metabolites.get_by_id(NAD)
product = FBA_model.metabolites.get_by_id(NADH)
for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if reactant in met_dict and product in met_dict:
        if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00211_c0
H2O_c0 + 2.0 NAD_c0 + UDP_glucose_c0 <=> 2.0 NADH_c0 + 3.0 H_c0 + UDPglucuronate_c0
0.0797027873221
rxn00501_c0
NAD_c0 + CoA_c0 + 3_Oxopropanoate_c0 <=> NADH_c0 + CO2_c0 + Acetyl_CoA_c0
18.3212231426
rxn01297_c0
H2O_c0 + NAD_c0 + HYXN_c0 <=> NADH_c0 + H_c0 + XAN_c0
0.647142353729
rxn00781_c0
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
33.1878716465
rxn00506_c0
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
14.0109382236
rxn02159_c0
NAD_c0 + L_Histidinol_c0 <=> NADH_c0 + H_c0 + L_Histidinal_c0
0.597844703793
rxn00499_c0
NAD_c0 + L_Lactate_c0 <=> NADH_c0 + Pyruvate_c0 + H_c0
996.102681298
rxn03062_c0
NAD_c0 + 3_Isopropylmalate_c0 <=> NADH_c0 + H_c0 + 2_isopropyl_3_oxosuccinate_c0
0.117487812423
rxn00863_c0
H2O_c0 + NAD_c0 + L_Histidinal_c0 --> NADH_c0 + 2.0 H_c0 + L_Histidine_c0
0.597844703793
rxn00908_c0
NAD_c0 + Glycine_c0 + Tetrahydrofolate_c0 <=> NADH_c0 + CO2_c0 + NH3_c0 + 5_10_Methylenete

In [83]:
## And what about that lactate?

product = FBA_model.metabolites.get_by_id('cpd00159_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if product in met_dict:
        if met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00145_c0
2.0 Cytochrome_c3_c0 + L_Lactate_c0 <=> Pyruvate_c0 + 2.0 H_c0 + 2.0 Cytochrome_c2_c0
-996.102681298


In [84]:
## Oh wow. There's the problem. The apparent correspondence between ATP synthase and fructose was a coincidence.
## The oxidative phosphorylation is being driven not by sugar oxidation, but lactate/pyruvate cycling--
## Pyruvate is being reduced instead of O2 as a terminal electron acceptor, 
## then the resulting lactate re-oxidized by NAD+ to generate NADH, which enters earlier in the chain.
## This is thermodynamically impossible, and since the lactate dehydrogenase reaction IS reversible,
## the solution is that reduced cytochrome C can't reduce pyruvate.

rxn = FBA_model.reactions.get_by_id('rxn00145_c0')
rxn.lower_bound = 0
solution = FBA_model.optimize()
print(solution.f)

0.007379887715008532


In [85]:
## What is the rate of citrate synthase now?
rxn_id = 'rxn00256_c0'
print(solution.x_dict[rxn_id])

5.68731046757


In [86]:
## check NTPs again

atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.id)
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.id)
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

rxn00364_c0
ATP_c0 + CMP_c0 + H_c0 <=> ADP_c0 + CDP_c0
-1.1768364061e-14
rxn05161_c0
H2O_c0 + ATP_c0 + L_Leucine_e0 --> ADP_c0 + Phosphate_c0 + H_c0 + L_Leucine_c0
-6.93889390391e-17
rxn00285_c0
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.597254312776
rxn01100_c0
ATP_c0 + 3_Phosphoglycerate_c0 <=> ADP_c0 + 1_3_Bisphospho_D_glycerate_c0
-33.1878716465
rxn00515_c0
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
rxn00409_c0
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-0.0326929025775
rxn05536_c0
H2O_c0 + ATP_c0 + met_L_ala_L_e0 --> ADP_c0 + Phosphate_c0 + H_c0 + met_L_ala_L_c0
-1.11022302463e-15
rxn_atp_syn
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
1001.79909978
rxn00237_c0
ATP_c0 + GDP_c0 <=> ADP_c0 + GTP_c0
865.653996424
rxn00712_c0
UTP_c0 + Uridine_c0 <=> UDP_c0 + H_c0 + UMP_c0
-1.05525014437
rxn00519_c0
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [87]:
## Where is NAD being reduced now?

NAD = 'cpd00003_c0'
NADH = 'cpd00004_c0'

reactant = FBA_model.metabolites.get_by_id(NAD)
product = FBA_model.metabolites.get_by_id(NADH)
for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if reactant in met_dict and product in met_dict:
        if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00211_c0
H2O_c0 + 2.0 NAD_c0 + UDP_glucose_c0 <=> 2.0 NADH_c0 + 3.0 H_c0 + UDPglucuronate_c0
0.0797027873221
rxn00501_c0
NAD_c0 + CoA_c0 + 3_Oxopropanoate_c0 <=> NADH_c0 + CO2_c0 + Acetyl_CoA_c0
107.630293455
rxn00068_c0
NAD_c0 + H_c0 + 2.0 Fe2_c0 <=> NADH_c0 + 2.0 fe3_c0
616.095084113
rxn00781_c0
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
33.1878716465
rxn00506_c0
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
14.0109382236
rxn02159_c0
NAD_c0 + L_Histidinol_c0 <=> NADH_c0 + H_c0 + L_Histidinal_c0
0.597844703793
rxn03062_c0
NAD_c0 + 3_Isopropylmalate_c0 <=> NADH_c0 + H_c0 + 2_isopropyl_3_oxosuccinate_c0
0.117487812423
rxn00863_c0
H2O_c0 + NAD_c0 + L_Histidinal_c0 --> NADH_c0 + 2.0 H_c0 + L_Histidine_c0
0.597844703793
rxn00908_c0
NAD_c0 + Glycine_c0 + Tetrahydrofolate_c0 <=> NADH_c0 + CO2_c0 + NH3_c0 + 5_10_Methylenetetrahydrofolate_c0
8.98796524811
rxn00371_c0
NAD_c0 + Formate_c0 --> NADH_c0 + CO2_c0

In [89]:
## Woah that's a lot of iron--more than even in the media, where I have it bounded (VERY generously) the same as ammonia!

product = FBA_model.metabolites.get_by_id('cpd10515_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if product in met_dict:
        if met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00056_c0
O2_c0 + 4.0 H_c0 + 4.0 Fe2_c0 <=> 2.0 H2O_c0 + 4.0 fe3_c0
-308.047597406


In [90]:
## Iron reaction with O2 oxidizes it, not the other way around. 
rxn = FBA_model.reactions.get_by_id('rxn00056_c0')
rxn.lower_bound = 0
solution = FBA_model.optimize()
print(solution.f)





0.006853773379760299


In [91]:
## Where is NAD being reduced now?

NAD = 'cpd00003_c0'
NADH = 'cpd00004_c0'

reactant = FBA_model.metabolites.get_by_id(NAD)
product = FBA_model.metabolites.get_by_id(NADH)
for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if reactant in met_dict and product in met_dict:
        if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00211_c0
H2O_c0 + 2.0 NAD_c0 + UDP_glucose_c0 <=> 2.0 NADH_c0 + 3.0 H_c0 + UDPglucuronate_c0
0.0740207525014
rxn00781_c0
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
33.6735102523
rxn00506_c0
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
8.38682540935
rxn02159_c0
NAD_c0 + L_Histidinol_c0 <=> NADH_c0 + H_c0 + L_Histidinal_c0
0.555224181494
rxn00499_c0
NAD_c0 + L_Lactate_c0 <=> NADH_c0 + Pyruvate_c0 + H_c0
0.00253589615051
rxn03062_c0
NAD_c0 + 3_Isopropylmalate_c0 <=> NADH_c0 + H_c0 + 2_isopropyl_3_oxosuccinate_c0
0.109112072206
rxn00863_c0
H2O_c0 + NAD_c0 + L_Histidinal_c0 --> NADH_c0 + 2.0 H_c0 + L_Histidine_c0
0.555224181494
rxn00908_c0
NAD_c0 + Glycine_c0 + Tetrahydrofolate_c0 <=> NADH_c0 + CO2_c0 + NH3_c0 + 5_10_Methylenetetrahydrofolate_c0
3.72447753003
rxn00834_c0
H2O_c0 + NAD_c0 + IMP_c0 <=> NADH_c0 + H_c0 + XMP_c0
0.601007387671
rxn01241_c0
NAD_c0 + Dihydrolipoamide_c0 <=> NADH_c0 + H_c0 + Lipo

In [92]:
## OK so it's acting as an acetate producer now. The growth rate is only slightly reduced, so where's the ATP from now?

atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.id)
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.id)
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

rxn00364_c0
ATP_c0 + CMP_c0 + H_c0 <=> ADP_c0 + CDP_c0
-6.93889390391e-18
rxn01673_c0
ATP_c0 + dCDP_c0 <=> ADP_c0 + dCTP_c0
-12.3579702433
rxn01219_c0
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-12.3883324594
rxn00285_c0
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.554675879624
rxn01100_c0
ATP_c0 + 3_Phosphoglycerate_c0 <=> ADP_c0 + 1_3_Bisphospho_D_glycerate_c0
-33.6735102523
rxn00515_c0
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
rxn00410_c0
ATP_c0 + NH3_c0 + UTP_c0 <=> ADP_c0 + Phosphate_c0 + CTP_c0 + 2.0 H_c0
-9.33250906028
rxn00409_c0
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-0.0303622160723
rxn_atp_syn
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
262.151797874
rxn00237_c0
ATP_c0 + GDP_c0 <=> ADP_c0 + GTP_c0
979.81159367
rxn00712_c0
UTP_c0 + Uridine_c0 <=> UDP_c0 + H_c0 + UMP_c0
-0.980021055572
rxn00519_c0
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [93]:
## It's still doing that weird thing with the PEP. Where is that coming from now?

product = FBA_model.metabolites.get_by_id('cpd00061_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if product in met_dict:
        if met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])


rxn00148_c0
ATP_c0 + Pyruvate_c0 <=> ADP_c0 + Phosphoenolpyruvate_c0 + H_c0
10.6737657643
rxn00305_c0
Oxaloacetate_c0 + GTP_c0 --> CO2_c0 + GDP_c0 + Phosphoenolpyruvate_c0
978.369559751
rxn00459_c0
2_Phospho_D_glycerate_c0 <=> H2O_c0 + Phosphoenolpyruvate_c0
33.6735102523


In [95]:
## Pyruvate kinase, the most irreversible reaction in glycolysis, is going backwards!

rxn = FBA_model.reactions.get_by_id('rxn00148_c0')
reverse(rxn)
rxn.lower_bound = 0
solution = FBA_model.optimize()
print(solution.f)


0.006853773379760294


In [96]:
print(rxn.lower_bound)
print(rxn.upper_bound)

0
1000.0


In [97]:
product = FBA_model.metabolites.get_by_id('cpd00061_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if product in met_dict:
        if met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00305_c0
Oxaloacetate_c0 + GTP_c0 --> CO2_c0 + GDP_c0 + Phosphoenolpyruvate_c0
989.043325515
rxn00459_c0
2_Phospho_D_glycerate_c0 <=> H2O_c0 + Phosphoenolpyruvate_c0
33.6735102523


In [98]:
## What is the rate of citrate synthase now?
rxn_id = 'rxn00256_c0'
print(solution.x_dict[rxn_id])

0.0


In [99]:
## check NTPs again

atp = 'cpd00002_c0'
gtp = 'cpd00038_c0'
ctp = 'cpd00052_c0'
utp = 'cpd00062_c0'
itp = 'cpd00068_c0'
adp = 'cpd00008_c0'
gdp = 'cpd00031_c0'
cdp = 'cpd00096_c0'
udp = 'cpd00014_c0'
idp = 'cpd00090_c0'

pairs = [(adp,atp),(gdp,gtp),(cdp,ctp),(udp,utp),(idp,itp)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.id)
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.id)
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

rxn01673_c0
ATP_c0 + dCDP_c0 <=> ADP_c0 + dCTP_c0
-12.3579702433
rxn01219_c0
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-12.3883324594
rxn00285_c0
ATP_c0 + CoA_c0 + Succinate_c0 <=> ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
-0.554675879624
rxn01100_c0
ATP_c0 + 3_Phosphoglycerate_c0 <=> ADP_c0 + 1_3_Bisphospho_D_glycerate_c0
-33.6735102523
rxn00515_c0
ATP_c0 + IDP_c0 <=> ADP_c0 + ITP_c0
-1000.0
rxn00409_c0
ATP_c0 + CDP_c0 <=> ADP_c0 + CTP_c0
-0.0303622160723
rxn_atp_syn
ADP_c0 + Phosphate_c0 + 4.0 H_e0 <=> H2O_c0 + ATP_c0 + 3.0 H_c0
262.151797874
rxn00237_c0
ATP_c0 + GDP_c0 <=> ADP_c0 + GTP_c0
990.485359435
rxn00712_c0
UTP_c0 + Uridine_c0 <=> UDP_c0 + H_c0 + UMP_c0
-0.980021055572
rxn00519_c0
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-1000.0


In [100]:
## The ATP synthase stoichiometry is very high for no pyruvate entering the TCA cycle
## Given that the reaction uses 4 ions/ATP and the stoichiometry is suggestively close to 1000/4, I suspect an ion leak

H_in = 'cpd00067_c0'
H_out = 'cpd00067_e0'
Na_in = 'cpd00971_c0'
Na_out = 'cpd00971_e0'

pairs =[(H_in,H_out), (Na_in,Na_out)]

for pair in pairs:
    reactant = FBA_model.metabolites.get_by_id(pair[0])
    product = FBA_model.metabolites.get_by_id(pair[1])
    for reaction in FBA_model.reactions:
        met_dict = reaction.metabolites
        if reactant in met_dict and product in met_dict:
            if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])
            elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
                print(reaction.build_reaction_string(use_metabolite_names = True))
                print(solution.x_dict[reaction.id])

0.5 O2_c0 + 2.5 H_c0 + Ubiquinol_8_c0 --> H2O_c0 + 2.5 H_e0 + Ubiquinone_8_c0
9.82237526878
NADH_c0 + 4.5 H_c0 + Ubiquinone_8_c0 <=> NAD_c0 + 3.5 H_e0 + Ubiquinol_8_c0
9.32259811393
H_e0 + Acetoacetate_e0 <=> H_c0 + Acetoacetate_c0
-3.13630202206
H_c0 + Na_e0 <=> H_e0 + Na_c0
20.8680950358
H_e0 + K_c0 <=> H_c0 + K_e0
-1000.0


In [102]:
## OK so there's a ginormous inward potassium flux--what's balancing it?

product = FBA_model.metabolites.get_by_id('cpd00205_e0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if product in met_dict:
        if met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn05206_c0
K_e0 <=> K_c0
-1000.0


In [103]:
## We can either have the leak, and then change the bounds on the antiport, or get rid of the leak
## Since we know the organism does take up K+ actively, I'll turn off the leak
## However, rather than deleting the reaction, I'll just set both bounds to zero
## That way, it's easy for others to put it back in if they want to simulate other conditions

rxn = FBA_model.reactions.get_by_id('rxn05206_c0')
rxn.lower_bound = 0
rxn.upper_bound = 0
solution = FBA_model.optimize()
print(solution.f)

0.0026746432701503543


In [104]:
## Now it grows sooo much slower!

## Where is NADH coming from now?

NAD = 'cpd00003_c0'
NADH = 'cpd00004_c0'

reactant = FBA_model.metabolites.get_by_id(NAD)
product = FBA_model.metabolites.get_by_id(NADH)
for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if reactant in met_dict and product in met_dict:
        if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00211_c0
H2O_c0 + 2.0 NAD_c0 + UDP_glucose_c0 <=> 2.0 NADH_c0 + 3.0 H_c0 + UDPglucuronate_c0
0.0288861473176
rxn01297_c0
H2O_c0 + NAD_c0 + HYXN_c0 <=> NADH_c0 + H_c0 + XAN_c0
1.03632537533e-17
rxn00781_c0
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
37.5311259521
rxn00506_c0
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
3.27290747682
rxn02159_c0
NAD_c0 + L_Histidinol_c0 <=> NADH_c0 + H_c0 + L_Histidinal_c0
0.216672851315
rxn00499_c0
NAD_c0 + L_Lactate_c0 <=> NADH_c0 + Pyruvate_c0 + H_c0
0.000989618009956
rxn03062_c0
NAD_c0 + 3_Isopropylmalate_c0 <=> NADH_c0 + H_c0 + 2_isopropyl_3_oxosuccinate_c0
0.0425803208608
rxn00863_c0
H2O_c0 + NAD_c0 + L_Histidinal_c0 --> NADH_c0 + 2.0 H_c0 + L_Histidine_c0
0.216672851315
rxn00908_c0
NAD_c0 + Glycine_c0 + Tetrahydrofolate_c0 <=> NADH_c0 + CO2_c0 + NH3_c0 + 5_10_Methylenetetrahydrofolate_c0
1.45345464587
rxn00834_c0
H2O_c0 + NAD_c0 + IMP_c0 <=> NADH_c0 + H_c0 + XM

In [105]:
## It's still not oxidizing AcCoA to CO2 by the TCA cycle--why?? Is there something missing?
## Yes, in fact malate dehydrogenase is!
## Without a way to recover carbon from malate, it can use neither the classic TCA cycle nor the glyoxylate shunt (if it has one)
## This would explain all the "creative" ways the model found to make oxaloacetate and NADH, 
## and why the citrate synthase reaction was so often unused.

stoich = {}
rxn = cobra.Reaction('rxn_00342_KEGG', name='Malate dehydrogenase')
malate = FBA_model.metabolites.get_by_id('cpd00130_c0')
NAD = FBA_model.metabolites.get_by_id('cpd00003_c0')
oxaloacetate = FBA_model.metabolites.get_by_id('cpd00032_c0')
NADH = FBA_model.metabolites.get_by_id('cpd00004_c0')
stoich[malate] = -1
stoich[NAD] = -1
stoich[oxaloacetate] = 1
stoich[NADH] = 1
rxn.add_metabolites(stoich)
## Assign gene
rxn.gene_reaction_rule = 'HVO_3007'
## Make it reversible, as we know that this step can go in both directions in most organisms
rxn.lower_bound = -1 * rxn.upper_bound
FBA_model.add_reaction(rxn)



In [107]:
solution = FBA_model.optimize()
print(solution.f)

0.002678306848621692


In [108]:
## That made a LOT less of a difference than I expected.

NAD = 'cpd00003_c0'
NADH = 'cpd00004_c0'

reactant = FBA_model.metabolites.get_by_id(NAD)
product = FBA_model.metabolites.get_by_id(NADH)
for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if reactant in met_dict and product in met_dict:
        if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00211_c0
H2O_c0 + 2.0 NAD_c0 + UDP_glucose_c0 <=> 2.0 NADH_c0 + 3.0 H_c0 + UDPglucuronate_c0
0.0289257139651
rxn00781_c0
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
37.5277442249
rxn00506_c0
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
3.27739052452
rxn02159_c0
NAD_c0 + L_Histidinol_c0 <=> NADH_c0 + H_c0 + L_Histidinal_c0
0.216969637807
rxn00499_c0
NAD_c0 + L_Lactate_c0 <=> NADH_c0 + Pyruvate_c0 + H_c0
0.00099097353399
rxn03062_c0
NAD_c0 + 3_Isopropylmalate_c0 <=> NADH_c0 + H_c0 + 2_isopropyl_3_oxosuccinate_c0
0.0426386450301
rxn00863_c0
H2O_c0 + NAD_c0 + L_Histidinal_c0 --> NADH_c0 + 2.0 H_c0 + L_Histidine_c0
0.216969637807
rxn00908_c0
NAD_c0 + Glycine_c0 + Tetrahydrofolate_c0 <=> NADH_c0 + CO2_c0 + NH3_c0 + 5_10_Methylenetetrahydrofolate_c0
1.45544550768
rxn00834_c0
H2O_c0 + NAD_c0 + IMP_c0 <=> NADH_c0 + H_c0 + XMP_c0
0.234860727556
rxn01241_c0
NAD_c0 + Dihydrolipoamide_c0 <=> NADH_c0 + H_c0 + Lip

In [109]:
## It's still turning more acetyl-CoA into acetate than it's oxidizing
## Plus the malate is not coming from isocitrate, obviously. 
## First check the immediate precursor

product = FBA_model.metabolites.get_by_id('cpd00130_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if product in met_dict:
        if met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])


rxn00799_c0
L_Malate_c0 <=> H2O_c0 + Fumarate_c0
-0.876636614622


In [110]:
## ...and the fumarate?

product = FBA_model.metabolites.get_by_id('cpd00106_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if product in met_dict:
        if met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn00800_c0
Adenylosuccinate_c0 <=> AMP_c0 + Fumarate_c0
0.336663170872
rxn00802_c0
L_Argininosuccinate_c0 <=> L_Arginine_c0 + Fumarate_c0
0.344698091418
rxn03136_c0
SAICAR_c0 <=> H_c0 + Fumarate_c0 + AICAR_c0
0.354554260621


In [111]:
## What is the rate of citrate synthase now?
rxn_id = 'rxn00256_c0'
print(solution.x_dict[rxn_id])

0.0


In [112]:
## Haloarchaea use ferredoxin as an electron carrier in oxidative phosphorylation,
## like many anaerobes but unlike most aerobes
## There is no way in the model for electrons to enter the transport chain from ferredoxin--
## Given that there is a probable ferredoxin:NAD reductase (HVO_A0618), I will assume they go to NAD, then complex I

stoich = {}
rxn = cobra.Reaction('rxn_05875_KEGG', name='Ferredoxin-NAD+ reductase')
red_ferredoxin = FBA_model.metabolites.get_by_id('cpd11620_c0')
NAD = FBA_model.metabolites.get_by_id('cpd00003_c0')
ox_ferredoxin = FBA_model.metabolites.get_by_id('cpd11621_c0')
NADH = FBA_model.metabolites.get_by_id('cpd00004_c0')
proton = FBA_model.metabolites.get_by_id('cpd00067_c0')
stoich[red_ferredoxin] = -1
stoich[NAD] = -1
stoich[ox_ferredoxin] = 1
stoich[NADH] = 1
stoich[proton] = 1
rxn.add_metabolites(stoich)
## Assign gene
rxn.gene_reaction_rule = 'HVO_A0618'
rxn.lower_bound = -1 * rxn.upper_bound
FBA_model.add_reaction(rxn)

In [115]:
## While I'm at it, I realized I left a proton out on malate dehydrogenase
mdh = FBA_model.reactions.get_by_id('rxn_00342_KEGG')
mdh.add_metabolites({'cpd00067_c0':1})
print(mdh.build_reaction_string(use_metabolite_names = True))


NAD_c0 + L_Malate_c0 <=> NADH_c0 + Oxaloacetate_c0 + H_c0


In [116]:
solution = FBA_model.optimize()
print(solution.f)

0.00786259847085565


In [117]:
## Whew that made it grow a lot better! Check NADH again:

NAD = 'cpd00003_c0'
NADH = 'cpd00004_c0'

reactant = FBA_model.metabolites.get_by_id(NAD)
product = FBA_model.metabolites.get_by_id(NADH)
for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if reactant in met_dict and product in met_dict:
        if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])



rxn00211_c0
H2O_c0 + 2.0 NAD_c0 + UDP_glucose_c0 <=> 2.0 NADH_c0 + 3.0 H_c0 + UDPglucuronate_c0
0.0849160634852
rxn00781_c0
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
32.7422974382
rxn00506_c0
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
9.62130449682
rxn02159_c0
NAD_c0 + L_Histidinol_c0 <=> NADH_c0 + H_c0 + L_Histidinal_c0
0.636949102124
rxn00499_c0
NAD_c0 + L_Lactate_c0 <=> NADH_c0 + Pyruvate_c0 + H_c0
0.00290916143422
rxn03062_c0
NAD_c0 + 3_Isopropylmalate_c0 <=> NADH_c0 + H_c0 + 2_isopropyl_3_oxosuccinate_c0
0.125172567656
rxn00863_c0
H2O_c0 + NAD_c0 + L_Histidinal_c0 --> NADH_c0 + 2.0 H_c0 + L_Histidine_c0
0.636949102124
rxn00908_c0
NAD_c0 + Glycine_c0 + Tetrahydrofolate_c0 <=> NADH_c0 + CO2_c0 + NH3_c0 + 5_10_Methylenetetrahydrofolate_c0
4.27269326103
rxn00834_c0
H2O_c0 + NAD_c0 + IMP_c0 <=> NADH_c0 + H_c0 + XMP_c0
0.689471259909
rxn01241_c0
NAD_c0 + Dihydrolipoamide_c0 <=> NADH_c0 + H_c0 + Lipo

In [118]:
## Now there is way TOO much NADH generated from ferredoxin (almost 100 per AcCoA) --what's going on?

product = FBA_model.metabolites.get_by_id('cpd11620_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if product in met_dict:
        if met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn05893_c0
2.0 H2O_c0 + NH3_c0 + 6.0 Oxidizedferredoxin_c0 <=> 8.0 H_c0 + Nitrite_c0 + 6.0 Reducedferredoxin_c0
40.9060747424
rxn05939_c0
CO2_c0 + H_c0 + Succinyl_CoA_c0 + Reducedferredoxin_c0 <=> CoA_c0 + 2_Oxoglutarate_c0 + Oxidizedferredoxin_c0
-2.4953528767


In [119]:
## Try temporarily disabling the ammonia oxidation, see what it does then
rxn = FBA_model.reactions.get_by_id('rxn05893_c0')
rxn.lower_bound = 0
rxn.upper_bound = 0
solution = FBA_model.optimize()
print(solution.f)

0.0044970757837720105


In [120]:
NAD = 'cpd00003_c0'
NADH = 'cpd00004_c0'

reactant = FBA_model.metabolites.get_by_id(NAD)
product = FBA_model.metabolites.get_by_id(NADH)
for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if reactant in met_dict and product in met_dict:
        if met_dict[reactant] < 0  and met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[reactant] > 0  and met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])


rxn00211_c0
H2O_c0 + 2.0 NAD_c0 + UDP_glucose_c0 <=> 2.0 NADH_c0 + 3.0 H_c0 + UDPglucuronate_c0
0.0485684184647
rxn00781_c0
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
35.8488992465
rxn00506_c0
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
5.50298169509
rxn02159_c0
NAD_c0 + L_Histidinol_c0 <=> NADH_c0 + H_c0 + L_Histidinal_c0
0.364308109243
rxn01301_c0
NAD_c0 + L_Homoserine_c0 <=> NADH_c0 + H_c0 + L_Aspartate4_semialdehyde_c0
9.10123687056
rxn00499_c0
NAD_c0 + L_Lactate_c0 <=> NADH_c0 + Pyruvate_c0 + H_c0
0.00166391804
rxn03062_c0
NAD_c0 + 3_Isopropylmalate_c0 <=> NADH_c0 + H_c0 + 2_isopropyl_3_oxosuccinate_c0
0.0715934464777
rxn00863_c0
H2O_c0 + NAD_c0 + L_Histidinal_c0 --> NADH_c0 + 2.0 H_c0 + L_Histidine_c0
0.364308109243
rxn00908_c0
NAD_c0 + Glycine_c0 + Tetrahydrofolate_c0 <=> NADH_c0 + CO2_c0 + NH3_c0 + 5_10_Methylenetetrahydrofolate_c0
2.44380092242
rxn00834_c0
H2O_c0 + NAD_c0 + IMP_c0 <=> NADH_c

In [121]:
product = FBA_model.metabolites.get_by_id('cpd11620_c0')

for reaction in FBA_model.reactions:
    met_dict = reaction.metabolites
    if product in met_dict:
        if met_dict[product] > 0 and solution.x_dict[reaction.id] > 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])
        elif met_dict[product] < 0 and solution.x_dict[reaction.id] < 0:
            print(reaction.id)
            print(reaction.build_reaction_string(use_metabolite_names = True))
            print(solution.x_dict[reaction.id])

rxn05939_c0
CO2_c0 + H_c0 + Succinyl_CoA_c0 + Reducedferredoxin_c0 <=> CoA_c0 + 2_Oxoglutarate_c0 + Oxidizedferredoxin_c0
-18.5489185309


In [122]:
cobra.io.write_sbml_model(FBA_model,'H_volcanii_curated.sbml')



In [4]:
FBA_model = cobra.io.read_sbml_model('H_volcanii_curated.sbml')

In [7]:
## add glucose-6-phosphate dehydrogenase reaction
stoich = {}
rxn = cobra.Reaction('rxn_00835_KEGG', name='Glucose-6-phosphate dehydrogenase (NAD)')
G6P = FBA_model.metabolites.get_by_id('cpd00079_c0')
PGL = cobra.Metabolite('cpd01236_KEGG',name='6_Phosphogluconolactone')
NAD = FBA_model.metabolites.get_by_id('cpd00003_c0')
NADH = FBA_model.metabolites.get_by_id('cpd00004_c0')
stoich[G6P] = -1
stoich[NAD] = -1
stoich[PGL] = 1
stoich[NADH] = 1
rxn.add_metabolites(stoich)
rxn.gene_reaction_rule = 'HVO_0511' 
FBA_model.add_reaction(rxn)
rxn.lower_bound = -1*rxn.upper_bound

In [8]:
## add 6-phosphogluconolactonase reaction
stoich = {}
rxn = cobra.Reaction('rxn_02035_KEGG', name='6-phosphogluconolactonase')
H2O = FBA_model.metabolites.get_by_id('cpd00001_c0')
PGA = FBA_model.metabolites.get_by_id('cpd00284_c0')
H = FBA_model.metabolites.get_by_id('cpd00067_c0')
stoich[PGL] = -1
stoich[H2O] = -1
stoich[PGA] = 1
stoich[H] = 1
rxn.add_metabolites(stoich)
rxn.gene_reaction_rule = 'Unknown' 
FBA_model.add_reaction(rxn)

In [4]:
def make_medium_from_components(FBA_model, med_components, default_bound):
    med = {}

    for component in med_components:
        try:
            met = FBA_model.metabolites.get_by_id(component)
        except:
            print('model to be filled has no ' + component + ': Adding it!')
            met = cobra.Metabolite(component)
        try:
            exchange = FBA_model.reactions.get_by_id('EX_' + component)
        except:
            exchange = FBA_model.add_boundary(met)
            print('new exchange reaction added: EX_' + component)
        exch_id = exchange.id
        med[exch_id] = default_bound
    return med

In [31]:
gluc_med_components = ['cpd00190_e0','cpd00013_e0','cpd00007_e0','cpd00009_e0','cpd00971_e0','cpd00048_e0','cpd10516_e0','cpd00149_e0','cpd00067_e0','cpd00001_e0','cpd00305_e0','cpd00104_e0']
fruc_med_components = ['cpd00082_e0','cpd00013_e0','cpd00007_e0','cpd00009_e0','cpd00971_e0','cpd00048_e0','cpd10516_e0','cpd00149_e0','cpd00067_e0','cpd00001_e0','cpd00305_e0','cpd00104_e0']

glucose_med = make_medium_from_components(FBA_model, gluc_med_components, 300)
fructose_med = make_medium_from_components(FBA_model, fruc_med_components, 300)

In [18]:
FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print(solution.f)

0.011092228101260404


In [20]:
glucose_med['EX_cpd00190_e0'] = 20
FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print(solution.f)

0.004440343812900169


In [21]:
fructose_med['EX_cpd00082_e0'] = 20
FBA_model.medium = fructose_med
solution = FBA_model.optimize()
print(solution.f)

0.004498769389385727


In [23]:
## Check that the model is using the pathways I assume it is

f_bis_aldolase = 'rxn_01068_KEGG'
KDGK = 'rxn01123_c0'

FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print('Glucose:')
print('fructose bisphosphate aldolase flux = ' + str(solution.x_dict[f_bis_aldolase]))
print('KDGK flux = ' + str(solution.x_dict[KDGK]))

FBA_model.medium = fructose_med
solution = FBA_model.optimize()
print('Fructose:')
print('fructose bisphosphate aldolase flux = ' + str(solution.x_dict[f_bis_aldolase]))
print('KDGK flux = ' + str(solution.x_dict[KDGK]))

Glucose:
fructose bisphosphate aldolase flux = 17.3654552089
KDGK flux = 4.32801710426e-14
Fructose:
fructose bisphosphate aldolase flux = 17.3307901459
KDGK flux = -1.08200427607e-14


In [24]:
## Hmmm... so it's using EMP for both of them, yet it does get more efficient growth on fructose somehow. 
## Found out it (the model--likely not the organism) has a 6-PFK, but it uses ADP rather than ATP
## What happens if that is turned off?

ADP_PFK = FBA_model.reactions.get_by_id('rxn04043_c0')
ADP_PFK.upper_bound = 0

f_bis_aldolase = 'rxn_01068_KEGG'
KDGK = 'rxn01123_c0'

FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print('Glucose:')
print('fructose bisphosphate aldolase flux = ' + str(solution.x_dict[f_bis_aldolase]))
print('KDGK flux = ' + str(solution.x_dict[KDGK]))

FBA_model.medium = fructose_med
solution = FBA_model.optimize()
print('Fructose:')
print('fructose bisphosphate aldolase flux = ' + str(solution.x_dict[f_bis_aldolase]))
print('KDGK flux = ' + str(solution.x_dict[KDGK]))


Glucose:
fructose bisphosphate aldolase flux = 2.20490292691e-13
KDGK flux = 17.4866906557
Fructose:
fructose bisphosphate aldolase flux = 17.3307901459
KDGK flux = -1.53509356667e-13


In [25]:
## OK, now it's using the "right" pathways.

FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print(solution.f)

FBA_model.medium = fructose_med
solution = FBA_model.optimize()
print(solution.f)

0.004236009816423325
0.004498769389385687


In [26]:
cobra.io.write_sbml_model(FBA_model,'H_volcanii_curated.sbml')

In [2]:
FBA_model = cobra.io.read_sbml_model('H_volcanii_curated.sbml')

In [6]:
## Correct NAD+ ferredoxin oxidoreductase stoichiometry
## It should be TWO ferredoxin (1-electron carrier) for every NAD (2-electron carrier)
rxn = FBA_model.reactions.get_by_id('rxn_05875_KEGG')
red_ferredoxin = FBA_model.metabolites.get_by_id('cpd11620_c0')
ox_ferredoxin = FBA_model.metabolites.get_by_id('cpd11621_c0')
proton = FBA_model.metabolites.get_by_id('cpd00067_c0')
stoich = {red_ferredoxin:-1, proton:-2, ox_ferredoxin:1}
rxn.add_metabolites(stoich)
print(rxn.build_reaction_string(use_metabolite_names = True))

NAD_c0 + H_c0 + 2.0 Reducedferredoxin_c0 <=> NADH_c0 + 2.0 Oxidizedferredoxin_c0


In [7]:
## Similarly, correct the ferredoxin-dependent ketoglutarate dehydrogenase
stoich = {red_ferredoxin:-1, ox_ferredoxin:1}
rxn = FBA_model.reactions.get_by_id('rxn05939_c0')
rxn.add_metabolites(stoich)
print(rxn.build_reaction_string(use_metabolite_names = True))

CO2_c0 + H_c0 + Succinyl_CoA_c0 + 2.0 Reducedferredoxin_c0 <=> CoA_c0 + 2_Oxoglutarate_c0 + 2.0 Oxidizedferredoxin_c0


In [13]:
## Take out NAD+ dependent pyruvate dehydrogenase
pyr_dh_nad = FBA_model.reactions.get_by_id('rxn00011_c0')
FBA_model.remove_reactions([pyr_dh_nad])


KeyError: 'rxn00011_c0'

In [16]:
PFOR = cobra.Reaction('rxn_01196_KEGG', name = 'Pyruvate:ferredoxin oxidoreductase')
stoich = {}
pyruvate = FBA_model.metabolites.get_by_id('cpd00020_c0')
coa = FBA_model.metabolites.get_by_id('cpd00010_c0')
accoa = FBA_model.metabolites.get_by_id('cpd00022_c0')
carbon_dioxide = FBA_model.metabolites.get_by_id('cpd00011_c0')
stoich[pyruvate] = -1
stoich[coa] = -1
stoich[ox_ferredoxin] = -2
stoich[red_ferredoxin] = 2
stoich[accoa] = 1
stoich[carbon_dioxide] = 1
stoich[proton] = 1
PFOR.add_metabolites(stoich)
PFOR.gene_reaction_rule = 'HVO_1304 and HVO_1305'
FBA_model.add_reaction(PFOR)

In [17]:
print(PFOR.build_reaction_string(use_metabolite_names = True))

CoA_c0 + Pyruvate_c0 + 2 Oxidizedferredoxin_c0 --> CO2_c0 + Acetyl_CoA_c0 + H_c0 + 2 Reducedferredoxin_c0


In [21]:
FBA_model.medium = glucose_med
glucose_med['EX_cpd00190_e0'] = 20
FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print(solution.f)

0.004346629334077283


In [22]:
fructose_med['EX_cpd00082_e0'] = 20
FBA_model.medium = fructose_med
solution = FBA_model.optimize()
print(solution.f)


0.004606423939687579


In [23]:
cobra.io.write_sbml_model(FBA_model,'H_volcanii_curated.sbml')

In [2]:
FBA_model = cobra.io.read_sbml_model('H_volcanii_curated.sbml')

In [37]:
## add genes and names for the automatically gap-filled reactions
import csv

names_genes_table = open('auto_gapfill_names_genes.csv','r')
reader = csv.reader(names_genes_table, dialect = 'excel')
for row in reader:
    if not row[0]=='':
        rxn_id = row[0]
        name = row[1]
        gene = row[2]
        if gene == 'Absent':
            gene = 'Unknown'
        rxn = FBA_model.reactions.get_by_id(rxn_id)
        rxn.name = name
        rxn.gene_reaction_rule = gene
names_genes_table.close()

In [38]:
## check that this worked
rxn = FBA_model.reactions.get_by_id('rxn00656_c')
print(rxn.name)
print(rxn.gene_reaction_rule)


Pantoate-beta-alanine ligase
Unknown


In [3]:
## The test biomass from the producibility-checking code is still in there. Take it out.
test = FBA_model.reactions.get_by_id('test_biomass')
FBA_model.remove_reactions([test])

In [50]:
## build CSV file with all reactions
import csv

csvfile = open('all_reactions.csv','w',newline ='')
rxn_writer = csv.writer(csvfile, dialect='excel')
title_row = ['Reaction ID','Name','Reaction equation','Gene']
rxn_writer.writerow(title_row)
for reaction in FBA_model.reactions:
    gene_string = reaction.gene_reaction_rule
    if gene_string == 'G_Unknown':
        gene = 'Unknown'
    elif 'fig' in gene_string:
        gene = 'From ModelSEED'
    else:
        gene = gene_string
    eqn = reaction.build_reaction_string(use_metabolite_names = True)
    row =[reaction.id, reaction.name, eqn, gene]
    rxn_writer.writerow(row)
csvfile.close()

In [6]:
glucose_med['EX_cpd00190_e0'] = 20
FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print(solution.f)

0.004346629334077371


In [10]:
## Find key fluxes
glucose_DH ='rxn01108_c0'
print(solution.x_dict[glucose_DH])
G6PDH = 'rxn_00835_KEGG'
print(solution.x_dict[G6PDH])
FBPase = 'rxn00549_c0'
print(solution.x_dict[FBPase])
aldolase = 'rxn_01068_KEGG'
print(solution.x_dict[aldolase])
PFOR = 'rxn_01196_KEGG'
print(solution.x_dict[PFOR])
citrate_syn = 'rxn00256_c0'
print(solution.x_dict[citrate_syn])
korB = 'rxn05939_c0'
print(solution.x_dict[korB])
MDH = 'rxn_00342_KEGG'
print(solution.x_dict[MDH])



17.4210578835
3.36211778991
0.0
0.0
10.6548490203
0.0
-18.145843169
18.9122408531


In [15]:
for reaction in FBA_model.reactions:
    if abs(solution.x_dict[reaction.id])> 5.0:
        print(reaction.build_reaction_string(use_metabolite_names = True))
        print(solution.x_dict[reaction.id])

Citrate_c0 <=> Isocitrate_c0
20.8392755021
H_c0 + Oxalosuccinate_c0 --> CO2_c0 + 2_Oxoglutarate_c0
21.7540671118
Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
-15.7230036679
GLCN_c0 --> H2O_c0 + 2_keto_3_deoxygluconate_c0
17.4210578835
ATP_c0 + L_Aspartate_c0 <=> ADP_c0 + 4_Phospho_L_aspartate_c0
6.70602320291
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
32.6598470435
H2O_c0 --> H2O_e0
89.6725674114
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
14.0081128407
NH3_e0 <=> NH3_c0
18.110013789
NAD_c0 + L_Homoserine_c0 <=> NADH_c0 + H_c0 + L_Aspartate4_semialdehyde_c0
-6.40149835176
NADPH_c0 + H_c0 + 4_Phospho_L_aspartate_c0 --> NADP_c0 + Phosphate_c0 + L_Aspartate4_semialdehyde_c0
6.70602320291
NAD_c0 + L_Lactate_c0 <=> NADH_c0 + Pyruvate_c0 + H_c0
-8.68762120429
NAD_c0 + beta_D_Glucose_c0 <=> NADH_c0 + H_c0 + Gluconolactone_c0
17.4210578835
0.5 O2_c0 + 2.5 H_c0 + Ubiquinol_8_c0 --> H2

In [16]:
## how is there a problem with PEPCK again? Reset it to irreversible:
pepck = FBA_model.reactions.get_by_id('rxn00519_c0')
print(pepck.build_reaction_string(use_metabolite_names = True))


Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0


In [17]:
pepck.lower_bound = 0

In [18]:
FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print(solution.f)

0.00434662933407737


In [19]:
for reaction in FBA_model.reactions:
    if abs(solution.x_dict[reaction.id])> 5.0:
        print(reaction.build_reaction_string(use_metabolite_names = True))
        print(solution.x_dict[reaction.id])

Citrate_c0 <=> Isocitrate_c0
20.8392755021
H_c0 + Oxalosuccinate_c0 --> CO2_c0 + 2_Oxoglutarate_c0
21.7540671118
Pyruvate_c0 + Malonyl_CoA_c0 <=> Acetyl_CoA_c0 + Oxaloacetate_c0
12.2948757344
NAD_c0 + CoA_c0 + 3_Oxopropanoate_c0 <=> NADH_c0 + CO2_c0 + Acetyl_CoA_c0
-12.2952234647
GLCN_c0 --> H2O_c0 + 2_keto_3_deoxygluconate_c0
17.4210578835
ATP_c0 + L_Aspartate_c0 <=> ADP_c0 + 4_Phospho_L_aspartate_c0
6.70602320291
ADP_c0 + Phosphoenolpyruvate_c0 + H_c0 --> ATP_c0 + Pyruvate_c0
15.7230036679
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
48.3828507114
H2O_c0 --> H2O_e0
89.6725674114
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
14.0081128407
NH3_e0 <=> NH3_c0
18.110013789
NAD_c0 + L_Homoserine_c0 <=> NADH_c0 + H_c0 + L_Aspartate4_semialdehyde_c0
-6.40149835176
NADPH_c0 + H_c0 + 4_Phospho_L_aspartate_c0 --> NADP_c0 + Phosphate_c0 + L_Aspartate4_semialdehyde_c0
6.70602320291
NAD_c0 + L_Lactate_c0 <=> NADH_c0

In [20]:
print(glucose_med)

{'EX_cpd00190_e0': 20, 'EX_cpd00013_e0': 100, 'EX_cpd00007_e0': 100, 'EX_cpd00009_e0': 100, 'EX_cpd00971_e0': 100, 'EX_cpd00048_e0': 100, 'EX_cpd10516_e0': 100, 'EX_cpd00149_e0': 100, 'EX_cpd00067_e0': 100, 'EX_cpd00001_e0': 100, 'EX_cpd00305_e0': 100, 'EX_cpd00104_e0': 100}


In [22]:
gluc_med_components = ['cpd00190_e0','cpd00013_e0','cpd00007_e0','cpd00009_e0','cpd00971_e0','cpd00048_e0','cpd10516_e0','cpd00149_e0','cpd00067_e0','cpd00001_e0','cpd00305_e0','cpd00104_e0']
fruc_med_components = ['cpd00082_e0','cpd00013_e0','cpd00007_e0','cpd00009_e0','cpd00971_e0','cpd00048_e0','cpd10516_e0','cpd00149_e0','cpd00067_e0','cpd00001_e0','cpd00305_e0','cpd00104_e0']

glucose_med = make_medium_from_components(FBA_model, gluc_med_components, 300)
fructose_med = make_medium_from_components(FBA_model, fruc_med_components, 300)

In [23]:
glucose_med['EX_cpd00190_e0'] = 20
FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print(solution.f)

0.00434662933407737


In [24]:
for reaction in FBA_model.reactions:
    if abs(solution.x_dict[reaction.id])> 5.0:
        print(reaction.build_reaction_string(use_metabolite_names = True))
        print(solution.x_dict[reaction.id])

Citrate_c0 <=> Isocitrate_c0
20.8392755021
H_c0 + Oxalosuccinate_c0 --> CO2_c0 + 2_Oxoglutarate_c0
21.7540671118
Pyruvate_c0 + Malonyl_CoA_c0 <=> Acetyl_CoA_c0 + Oxaloacetate_c0
12.2948757344
NAD_c0 + CoA_c0 + 3_Oxopropanoate_c0 <=> NADH_c0 + CO2_c0 + Acetyl_CoA_c0
-12.2952234647
GLCN_c0 --> H2O_c0 + 2_keto_3_deoxygluconate_c0
17.4210578835
ATP_c0 + L_Aspartate_c0 <=> ADP_c0 + 4_Phospho_L_aspartate_c0
6.70602320291
ADP_c0 + Phosphoenolpyruvate_c0 + H_c0 --> ATP_c0 + Pyruvate_c0
15.7230036679
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
48.3828507114
H2O_c0 --> H2O_e0
89.6725674114
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
14.0081128407
NH3_e0 <=> NH3_c0
18.110013789
NAD_c0 + L_Homoserine_c0 <=> NADH_c0 + H_c0 + L_Aspartate4_semialdehyde_c0
-6.40149835176
NADPH_c0 + H_c0 + 4_Phospho_L_aspartate_c0 --> NADP_c0 + Phosphate_c0 + L_Aspartate4_semialdehyde_c0
6.70602320291
NAD_c0 + L_Lactate_c0 <=> NADH_c0

In [25]:
## Try setting citrate lyase as irreversible
citL = FBA_model.reactions.get_by_id('rxn00265_c0')
citL.lower_bound = 0

In [26]:
solution = FBA_model.optimize()
print(solution.f)

0.004218229258123782


In [27]:
for reaction in FBA_model.reactions:
    if abs(solution.x_dict[reaction.id])> 5.0:
        print(reaction.build_reaction_string(use_metabolite_names = True))
        print(solution.x_dict[reaction.id])

ATP_c0 + dCDP_c0 <=> ADP_c0 + dCTP_c0
-10.8566253708
Citrate_c0 <=> Isocitrate_c0
21.4052861489
H_c0 + Oxalosuccinate_c0 --> CO2_c0 + 2_Oxoglutarate_c0
22.2930546786
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-10.8753121264
Pyruvate_c0 + Malonyl_CoA_c0 <=> Acetyl_CoA_c0 + Oxaloacetate_c0
11.9316832795
H2O_c0 + dCMP_c0 --> Phosphate_c0 + Deoxycytidine_c0
10.8753121264
NAD_c0 + CoA_c0 + 3_Oxopropanoate_c0 <=> NADH_c0 + CO2_c0 + Acetyl_CoA_c0
-11.9320207379
GLCN_c0 --> H2O_c0 + 2_keto_3_deoxygluconate_c0
17.4972402166
ATP_c0 + L_Aspartate_c0 <=> ADP_c0 + 4_Phospho_L_aspartate_c0
6.50792628173
ADP_c0 + Phosphoenolpyruvate_c0 + H_c0 --> ATP_c0 + Pyruvate_c0
15.8493467746
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
48.1601475714
H2O_c0 --> H2O_e0
90.5684442733
ATP_c0 + AMP_c0 + H_c0 <=> 2.0 ADP_c0
9.15688989124
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
5.16176277858
NH3_e0 <=> NH3_c0
17.5750412925
NAD_c

In [28]:
glucose_DH ='rxn01108_c0'
print(solution.x_dict[glucose_DH])
G6PDH = 'rxn_00835_KEGG'
print(solution.x_dict[G6PDH])
FBPase = 'rxn00549_c0'
print(solution.x_dict[FBPase])
aldolase = 'rxn_01068_KEGG'
print(solution.x_dict[aldolase])
PFOR = 'rxn_01196_KEGG'
print(solution.x_dict[PFOR])
citrate_syn = 'rxn00256_c0'
print(solution.x_dict[citrate_syn])
korB = 'rxn05939_c0'
print(solution.x_dict[korB])
MDH = 'rxn_00342_KEGG'
print(solution.x_dict[MDH])

17.4972402166
3.26280033116
0.0
0.0
19.9542574664
21.4052861489
-18.7914182068
19.5351763896


In [32]:
fructose_med['EX_cpd00082_e0'] = 20
FBA_model.medium = fructose_med
solution = FBA_model.optimize()
print(solution.f)

0.004471095547160047


In [33]:
glucose_DH ='rxn01108_c0'
print(solution.x_dict[glucose_DH])
G6PDH = 'rxn_00835_KEGG'
print(solution.x_dict[G6PDH])
FBPase = 'rxn00549_c0'
print(solution.x_dict[FBPase])
aldolase = 'rxn_01068_KEGG'
print(solution.x_dict[aldolase])
PFOR = 'rxn_01196_KEGG'
print(solution.x_dict[PFOR])
citrate_syn = 'rxn00256_c0'
print(solution.x_dict[citrate_syn])
korB = 'rxn05939_c0'
print(solution.x_dict[korB])
MDH = 'rxn_00342_KEGG'
print(solution.x_dict[MDH])

0.0
0.0
0.0
18.5000070585
19.9053913422
21.4434034995
-18.6728444327
19.4611879996


In [34]:
for reaction in FBA_model.reactions:
    if abs(solution.x_dict[reaction.id])> 5.0:
        print(reaction.build_reaction_string(use_metabolite_names = True))
        print(solution.x_dict[reaction.id])

ATP_c0 + dCDP_c0 <=> ADP_c0 + dCTP_c0
-11.5074374535
Citrate_c0 <=> Isocitrate_c0
21.4434034995
H_c0 + Oxalosuccinate_c0 --> CO2_c0 + 2_Oxoglutarate_c0
22.3843902684
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-11.5272444068
Pyruvate_c0 + Malonyl_CoA_c0 <=> Acetyl_CoA_c0 + Oxaloacetate_c0
12.6469408647
H2O_c0 + dCMP_c0 --> Phosphate_c0 + Deoxycytidine_c0
11.5272444068
NAD_c0 + CoA_c0 + 3_Oxopropanoate_c0 <=> NADH_c0 + CO2_c0 + Acetyl_CoA_c0
-12.6472985523
ATP_c0 + L_Aspartate_c0 <=> ADP_c0 + 4_Phospho_L_aspartate_c0
6.89805092111
ADP_c0 + Phosphoenolpyruvate_c0 + H_c0 --> ATP_c0 + Pyruvate_c0
14.100538462
Phosphoenolpyruvate_c0 + D_Fructose_e0 <=> Pyruvate_c0 + D_fructose_1_phosphate_c0
20.0
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
63.6448070067
H2O_c0 --> H2O_e0
88.8041380155
L_Glutamate_e0 + Na_e0 <=> L_Glutamate_c0 + Na_c0
13.6134128782
ATP_c0 + AMP_c0 + H_c0 <=> 2.0 ADP_c0
8.20581656135
ATP_c0 + D_fructose_1_phosphate_

In [36]:
## Uh-oh, the "bad" PFK is still in there, just in reverse. Remove it.
bad_PFK = FBA_model.reactions.get_by_id('rxn04043_c0')
FBA_model.remove_reactions([bad_PFK])

In [37]:
FBA_model.medium = fructose_med
solution = FBA_model.optimize()
print(solution.f)

0.00445459711695636


In [38]:
glucose_DH ='rxn01108_c0'
print(solution.x_dict[glucose_DH])
G6PDH = 'rxn_00835_KEGG'
print(solution.x_dict[G6PDH])
FBPase = 'rxn00549_c0'
print(solution.x_dict[FBPase])
aldolase = 'rxn_01068_KEGG'
print(solution.x_dict[aldolase])
PFOR = 'rxn_01196_KEGG'
print(solution.x_dict[PFOR])
citrate_syn = 'rxn00256_c0'
print(solution.x_dict[citrate_syn])
korB = 'rxn05939_c0'
print(solution.x_dict[korB])
MDH = 'rxn_00342_KEGG'
print(solution.x_dict[MDH])

0.0
3.00143219034e-16
1.49445793811
18.5055420619
19.979540842
21.5118777043
-18.7515420548
19.5369766185


In [39]:
for reaction in FBA_model.reactions:
    if abs(solution.x_dict[reaction.id])> 5.0:
        print(reaction.build_reaction_string(use_metabolite_names = True))
        print(solution.x_dict[reaction.id])

ATP_c0 + dCDP_c0 <=> ADP_c0 + dCTP_c0
-9.95963278006
Citrate_c0 <=> Isocitrate_c0
21.5118777043
H_c0 + Oxalosuccinate_c0 --> CO2_c0 + 2_Oxoglutarate_c0
22.4493922135
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-11.5733105857
Pyruvate_c0 + Malonyl_CoA_c0 <=> Acetyl_CoA_c0 + Oxaloacetate_c0
12.600273405
H2O_c0 + dCMP_c0 --> Phosphate_c0 + Deoxycytidine_c0
11.5733105857
NAD_c0 + CoA_c0 + 3_Oxopropanoate_c0 <=> NADH_c0 + CO2_c0 + Acetyl_CoA_c0
-12.6006297728
ATP_c0 + L_Aspartate_c0 <=> ADP_c0 + 4_Phospho_L_aspartate_c0
6.87259697801
ADP_c0 + Phosphoenolpyruvate_c0 + H_c0 --> ATP_c0 + Pyruvate_c0
14.1223075907
Phosphoenolpyruvate_c0 + D_Fructose_e0 <=> Pyruvate_c0 + D_fructose_1_phosphate_c0
20.0
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
36.6486032906
H2O_c0 --> H2O_e0
88.9192513577
ATP_c0 + AMP_c0 + H_c0 <=> 2.0 ADP_c0
9.7582404444
ATP_c0 + D_fructose_1_phosphate_c0 <=> ADP_c0 + H_c0 + D_fructose_1_6_bisphosphate_c0
20.0
H2O_c0

In [42]:
glucose_med['EX_cpd00190_e0'] = 20
FBA_model.medium = glucose_med
solution = FBA_model.optimize()
print(solution.f)

0.004218229258123807


In [43]:
glucose_DH ='rxn01108_c0'
print(solution.x_dict[glucose_DH])
G6PDH = 'rxn_00835_KEGG'
print(solution.x_dict[G6PDH])
FBPase = 'rxn00549_c0'
print(solution.x_dict[FBPase])
aldolase = 'rxn_01068_KEGG'
print(solution.x_dict[aldolase])
PFOR = 'rxn_01196_KEGG'
print(solution.x_dict[PFOR])
citrate_syn = 'rxn00256_c0'
print(solution.x_dict[citrate_syn])
korB = 'rxn05939_c0'
print(solution.x_dict[korB])
MDH = 'rxn_00342_KEGG'
print(solution.x_dict[MDH])

0.0
3.26280033116
0.0
0.0
19.9542574664
21.4052861489
-18.7914182068
19.5351763896


In [44]:
cobra.io.write_sbml_model(FBA_model,'H_volcanii_curated.sbml')

In [45]:
for reaction in FBA_model.reactions:
    if abs(solution.x_dict[reaction.id])> 5.0:
        print(reaction.build_reaction_string(use_metabolite_names = True))
        print(solution.x_dict[reaction.id])

ATP_c0 + dCDP_c0 <=> ADP_c0 + dCTP_c0
-9.69720287691
Citrate_c0 <=> Isocitrate_c0
21.4052861489
H_c0 + Oxalosuccinate_c0 --> CO2_c0 + 2_Oxoglutarate_c0
22.2930546786
ATP_c0 + H_c0 + dCMP_c0 <=> ADP_c0 + dCDP_c0
-11.2252564257
Pyruvate_c0 + Malonyl_CoA_c0 <=> Acetyl_CoA_c0 + Oxaloacetate_c0
11.9316832795
H2O_c0 + dCMP_c0 --> Phosphate_c0 + Deoxycytidine_c0
11.2252564257
NAD_c0 + CoA_c0 + 3_Oxopropanoate_c0 <=> NADH_c0 + CO2_c0 + Acetyl_CoA_c0
-11.9320207379
GLCN_c0 --> H2O_c0 + 2_keto_3_deoxygluconate_c0
17.4972402166
ATP_c0 + L_Aspartate_c0 <=> ADP_c0 + 4_Phospho_L_aspartate_c0
6.50792628172
ADP_c0 + Phosphoenolpyruvate_c0 + H_c0 --> ATP_c0 + Pyruvate_c0
15.8493467746
NAD_c0 + Phosphate_c0 + Glyceraldehyde3_phosphate_c0 <=> NADH_c0 + H_c0 + 1_3_Bisphospho_D_glycerate_c0
36.7736956774
H2O_c0 --> H2O_e0
90.5684442733
ATP_c0 + AMP_c0 + H_c0 <=> 2.0 ADP_c0
9.50649673215
H2O_c0 + NAD_c0 + Acetaldehyde_c0 --> NADH_c0 + Acetate_c0 + 2.0 H_c0
5.16176277858
NH3_e0 <=> NH3_c0
17.5750412925
NADPH